# Assessing People’s Skills

In [10]:
# Activate local environment, see `Project.toml`
import Pkg; Pkg.activate("."); Pkg.instantiate(); 

  Activating project at `~/repos/RxInfer.jl/examples`


┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-aarch64-4.0/build/default-macmini-aarch64-4-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/API.jl:1535


The goal of this demo is to demonstrate the use of the `@node` and `@rule` macros, which allow a user to define custom factor nodes and associated update rules respectively. We will introduce these macros in the context of a root cause analysis on a student's test results. This demo is inspired by Chapter 2 of "Model-Based Machine Learning" by Winn et al.
We consider a student who takes a test that consists of three questions. Answering each question correctly requires a combination of skills and attitude. More precisely, has the student studied for the test, and (since students like to party) have they slept the night before?
We consider a model of binary variables. More precisely, we model the results for question $i$ as $r_i\in\{0,1\}$, and root causes $s_i \in \{0, 1\}$, where $s_1$ represents whether the student has slept or not, and $s_2$ and $s_3$ represent whether the student as studied the chapters for the corresponding questions.
We assume the following model:
- If the student as slept, they will be able to answer the first question;
- If the student has slept or studied chapter two, then they will be able to answer question two;
- If the student has answered question two correctly and studied chapter three, then they will be able to answer question three.

In [2]:
using RxInfer, Random

GRAPH HERE

In RxInfer, the `@node` macro is utilized to depict a factor from Forney-style Factor Graph (FFG). This involves defining a function (referred to as a form) and the random variables, or edges, upon which the function operates in the FFG notation.

There exist two types of edges in FFG notation: one for computing a posterior marginal, and the other called half-edges, which correspond to likelihoods in RxInfer. These half-edges are represented by constant PointMass marginals and do not require inference.

In order to implement this functionality in RxInfer, one must specify a formtype identifier (function identifier), which in our case is a structure `AddNoise`, and provide a list of interfaces that correspond to the random variables in FFG notation.

In [3]:
# Create AddNoise node
struct AddNoise end

@node AddNoise Stochastic [out, in]

In [4]:
# Adding update rule for AddNoise node
@rule AddNoise(:in, Marginalisation) (q_out::PointMass,) = begin     
    return Bernoulli(mean(q_out))
end

In [5]:
# GraphPPL.jl export `@model` macro for model specification
# It accepts a regular Julia function and builds an FFG under the hood
@model function skill_model()

    res = datavar(Float64, 3)

    laziness ~ Bernoulli(0.5)
    skill2 ~ Bernoulli(0.5)
    skill3 ~ Bernoulli(0.5)

    test1 ~ ¬laziness
    test2 ~ ¬laziness -> skill2
    test3 ~ test2 && skill3
    
    res[1] ~ AddNoise(test1)
    res[2] ~ AddNoise(test2)
    res[3] ~ AddNoise(test3)

end

Let us assume that a student scoared $70\%$ and $95\%$ at first and second tests respectively. But got only $30\%$ on the third one. 

In [6]:
test_results = [0.7, 0.95, 0.3]

inference_result = inference(
    model = skill_model(),
    data  = (res = test_results, )
)

Inference results:
  Posteriors       | available for (skill3, test2, test3, skill2, laziness)


In [7]:
inference_result.posteriors[:laziness]

Bernoulli{Float64}(p=0.18704156479217607)

In [8]:
inference_result.posteriors[:skill2]

Bernoulli{Float64}(p=0.5806845965770171)

In [9]:
inference_result.posteriors[:skill3]

Bernoulli{Float64}(p=0.3025672371638141)

The results make sense. On the one hand, the student answered the first question correctly, which immediately gives us reason to believe that he is not lazy. He answered the second question pretty well, but this does not mean that the student had the skills to answer this question (attendance,i.e., lack of laziness, could help). To answer the third question, it was necessary to answer the second and have additional skills (#3). Unfortunately, the student's answer was weak, so our confidence about skill #3 was shattered.